<a href="https://colab.research.google.com/github/Prashitraj/Algorithmic-Trading/blob/main/Quantative_Momentum_Investing_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score

In [ ]:
stocks = pd.read_csv('/content/drive/MyDrive/StockAnalysis/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [ ]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'companyName': 'Apple Inc', 'marketcap': 2068436237360, 'week52high': 144.93, 'week52low': 61.16, 'week52highSplitAdjustOnly': 151.59, 'week52lowSplitAdjustOnly': 60.52, 'week52change': 0.998800326976871, 'sharesOutstanding': 17550755368, 'float': 0, 'avg10Volume': 114894171, 'avg30Volume': 121741741, 'day200MovingAvg': 126.77, 'day50MovingAvg': 130.41, 'employees': 152325, 'ttmEPS': 3.72, 'ttmDividendRate': 0.8512178190260867, 'dividendYield': 0.006951109668950046, 'nextDividendDate': '0', 'exDividendDate': '2021-01-29', 'nextEarningsDate': '2021-01-17', 'peRatio': 32.46522926009759, 'beta': 1.2116893877584514, 'maxChangePercent': 45.60913367187312, 'year5ChangePercent': 4.049263382859919, 'year2ChangePercent': 1.7165196754979029, 'year1ChangePercent': 0.9148669469179178, 'ytdChangePercent': -0.08695886090342658, 'month6ChangePercent': 0.0852659360551815, 'month3ChangePercent': -0.08127780505556187, 'month1ChangePercent': -0.0004262318891577606, 'day30ChangePercent': -0.0345113583767

In [ ]:
data['year1ChangePercent']

0.9148669469179178

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
my_columns = ['Ticker', 'Stock Price', 'One Year Price Return', 'Number Of Shares to Buy']
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(pd.Series([symbol,data[symbol]['price'],data[symbol]['stats']['year1ChangePercent'],'N/A'],index=my_columns),ignore_index=True)
final_dataframe

,Ticker,Stock Price,One Year Price Return,Number Of Shares to Buy
0,A,127.96,0.751216,N/A
1,AAL,23.24,0.472272,N/A
2,AAP,187.59,0.957804,N/A
3,AAPL,122.40,0.929876,N/A
4,ABBV,107.31,0.512861,N/A
...,...,...,...,...
500,YUM,110.97,0.464892,N/A
501,ZBH,165.17,0.618632,N/A
502,ZBRA,479.47,1.43541,N/A
503,ZION,56.51,1.13959,N/A


In [ ]:
final_dataframe.sort_values('One Year Price Return',ascending = False, inplace = True )
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Stock Price,One Year Price Return,Number Of Shares to Buy
0,179,FCX,34.030,3.88812,N/A
1,275,LB,63.100,3.74112,N/A
2,467,VIAC,48.320,2.63268,N/A
3,208,GPS,30.660,2.61234,N/A
4,462,URI,341.680,2.43596,N/A
5,148,DVN,23.550,2.37484,N/A
6,272,KSS,59.900,2.36839,N/A
7,212,HAL,22.130,2.1349,N/A
8,175,FANG,75.940,2.03358,N/A
9,314,MOS,31.440,1.94578,N/A


In [ ]:
portfolio_size = input('Enter your portfolio size:')

try:
    val = float(portfolio_size)
except ValueError:
    print('Please enter an integer')

Enter your portfolio size:1000000


In [ ]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number Of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price'])
final_dataframe

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,index,Ticker,Stock Price,One Year Price Return,Number Of Shares to Buy
0,179,FCX,34.030,3.88812,587
1,275,LB,63.100,3.74112,316
2,467,VIAC,48.320,2.63268,413
3,208,GPS,30.660,2.61234,652
4,462,URI,341.680,2.43596,58
5,148,DVN,23.550,2.37484,849
6,272,KSS,59.900,2.36839,333
7,212,HAL,22.130,2.1349,903
8,175,FANG,75.940,2.03358,263
9,314,MOS,31.440,1.94578,636


In [ ]:
hqm_columns = ['Ticker', 'Price', 'Number of Shares to Buy',
               'One Year Price Return', 'One Year Return Percentile',
               'Six Month Price Return', 'Six Month Return Percentile',
               'Three Month Price Return', 'Three Month Return Percentile',
               'One Month Price Return', 'One Month Return Percentile',
               'HQM Score']

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(pd.Series(
            [symbol,data[symbol]['price'],'N/A',
             data[symbol]['stats']['year1ChangePercent'],'N/A',
             data[symbol]['stats']['month6ChangePercent'],'N/A',
             data[symbol]['stats']['month3ChangePercent'],'N/A',
             data[symbol]['stats']['month1ChangePercent'],'N/A',
             'N/A'],index=hqm_columns),ignore_index=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,A,131.80,N/A,0.752116,N/A,0.298726,N/A,0.0737002,N/A,0.0296211,N/A,N/A
1,AAL,23.64,N/A,0.470968,N/A,0.886346,N/A,0.472836,N/A,0.0997706,N/A,N/A
2,AAP,192.98,N/A,0.96815,N/A,0.266234,N/A,0.174841,N/A,0.178183,N/A,N/A
3,AAPL,125.37,N/A,0.915512,N/A,0.0858227,N/A,-0.0824003,N/A,-0.000418656,N/A,N/A
4,ABBV,108.04,N/A,0.533188,N/A,0.262651,N/A,0.0384744,N/A,-0.0169699,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.18,N/A,0.459769,N/A,0.195501,N/A,0.0100106,N/A,0.0447667,N/A,N/A
501,ZBH,167.62,N/A,0.626067,N/A,0.205417,N/A,0.0837398,N/A,-0.0092538,N/A,N/A
502,ZBRA,484.03,N/A,1.41916,N/A,0.854286,N/A,0.271013,N/A,-0.0450359,N/A,N/A
503,ZION,56.01,N/A,1.13279,N/A,1.0408,N/A,0.306986,N/A,0.0516472,N/A,N/A


In [ ]:
hqm_dataframe = hqm_dataframe.fillna(value=np.nan)

In [ ]:
timeperiods = ['One Year', 'Six Month','Three Month','One Month']

for row in hqm_dataframe.index:
    for timeperiod in timeperiods:
        change_col = f'{timeperiod} Price Return'
        percentile_col = f'{timeperiod} Return Percentile'
        hqm_dataframe.loc[row,percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row,change_col])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,A,131.80,N/A,0.752116,64.1584,0.298726,49.901,0.073700,35.8416,0.029621,27.7228,N/A
1,AAL,23.64,N/A,0.470968,37.2277,0.886346,91.6832,0.472836,97.0297,0.099771,66.3366,N/A
2,AAP,192.98,N/A,0.968150,77.6238,0.266234,45.1485,0.174841,62.1782,0.178183,95.6436,N/A
3,AAPL,125.37,N/A,0.915512,74.4554,0.085823,18.2178,-0.082400,3.56436,-0.000419,15.8416,N/A
4,ABBV,108.04,N/A,0.533188,44.1584,0.262651,44.7525,0.038474,24.9505,-0.016970,9.90099,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.18,N/A,0.459769,35.8416,0.195501,35.6436,0.010011,17.6238,0.044767,35.8416,N/A
501,ZBH,167.62,N/A,0.626067,53.0693,0.205417,37.6238,0.083740,39.4059,-0.009254,12.6733,N/A
502,ZBRA,484.03,N/A,1.419160,91.4851,0.854286,90.6931,0.271013,82.5743,-0.045036,5.94059,N/A
503,ZION,56.01,N/A,1.132786,85.1485,1.040796,96.0396,0.306986,86.5347,0.051647,39.0099,N/A


In [ ]:
from statistics import mean 
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for timeperiod in timeperiods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{timeperiod} Return Percentile'])
        hqm_dataframe.loc[row,'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

In [ ]:
hqm_dataframe.sort_values('HQM Score',ascending=False,inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True)
hqm_dataframe

,index,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,208,GPS,30.180,N/A,2.536651,98.6139,0.830482,89.901,0.459347,96.4356,0.189722,96.6337,95.396
1,344,NUE,80.500,N/A,1.490181,92.8713,0.802145,88.7129,0.540019,98.4158,0.333081,99.0099,94.7525
2,275,LB,63.000,N/A,3.817439,99.2079,1.050575,96.2376,0.588256,98.6139,0.130602,83.5644,94.4059
3,352,OKE,52.970,N/A,1.421849,91.6832,1.164264,97.0297,0.354384,91.0891,0.162754,92.4752,93.0693
4,220,HIG,70.000,N/A,1.021672,80.9901,0.940510,93.8614,0.415639,94.8515,0.348379,99.2079,92.2277
5,160,EOG,77.630,N/A,1.003932,79.4059,1.040152,95.8416,0.527714,97.8218,0.161414,92.0792,91.2871
6,175,FANG,78.260,N/A,2.028616,97.6238,1.633462,98.2178,0.603495,99.0099,0.104451,68.7129,90.8911
7,462,URI,331.990,N/A,2.337536,98.2178,0.964123,94.6535,0.418526,95.0495,0.113374,74.0594,90.495
8,306,MHK,198.830,N/A,1.441963,92.2772,1.126668,96.6337,0.412808,94.4554,0.119951,78.6139,90.495
9,253,IVZ,25.730,N/A,1.682031,95.0495,1.357261,97.8218,0.425476,95.4455,0.107249,70.297,89.6535


In [ ]:
portfolio_size = input('Enter your portfolio size:')

try:
    val = float(portfolio_size)
except ValueError:
    print('Please enter an integer')

Enter your portfolio size:1000000


In [ ]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in range(0,len(final_dataframe)):
    hqm_dataframe.loc[i,'Number Of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i,'Price'])
hqm_dataframe

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,index,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score,Number Of Shares to Buy
0,208,GPS,30.180,N/A,2.536651,98.6139,0.830482,89.901,0.459347,96.4356,0.189722,96.6337,95.396,662.0
1,344,NUE,80.500,N/A,1.490181,92.8713,0.802145,88.7129,0.540019,98.4158,0.333081,99.0099,94.7525,248.0
2,275,LB,63.000,N/A,3.817439,99.2079,1.050575,96.2376,0.588256,98.6139,0.130602,83.5644,94.4059,317.0
3,352,OKE,52.970,N/A,1.421849,91.6832,1.164264,97.0297,0.354384,91.0891,0.162754,92.4752,93.0693,377.0
4,220,HIG,70.000,N/A,1.021672,80.9901,0.940510,93.8614,0.415639,94.8515,0.348379,99.2079,92.2277,285.0
5,160,EOG,77.630,N/A,1.003932,79.4059,1.040152,95.8416,0.527714,97.8218,0.161414,92.0792,91.2871,257.0
6,175,FANG,78.260,N/A,2.028616,97.6238,1.633462,98.2178,0.603495,99.0099,0.104451,68.7129,90.8911,255.0
7,462,URI,331.990,N/A,2.337536,98.2178,0.964123,94.6535,0.418526,95.0495,0.113374,74.0594,90.495,60.0
8,306,MHK,198.830,N/A,1.441963,92.2772,1.126668,96.6337,0.412808,94.4554,0.119951,78.6139,90.495,100.0
9,253,IVZ,25.730,N/A,1.682031,95.0495,1.357261,97.8218,0.425476,95.4455,0.107249,70.297,89.6535,777.0
